# Collateral Value

In [1]:
from mantle import mantle
from env import *
from numpy import array, sum
from json import dumps

user_wallet_address = 'terra12hnhh5vtyg5juqnzm43970nh4fw42pt27nw9g9'

result = mantle(
    wasm_query={
        'overseer_collaterals': {
            'contractAddress': moneymarket_overseer_addr,
            'query': {
                'collaterals': {
                    'borrower': user_wallet_address
                }
            }
        },
        'oracle_prices': {
            'contractAddress': moneymarket_oracle_addr,
            'query': {
                'prices': {}
            }
        }
    }
)

print(dumps(result, indent=2))

{
  "oracle_prices": {
    "Ok": {
      "prices": [
        {
          "asset": "terra19mkj9nec6e3y5754tlnuz4vem7lzh4n0lc2s3l",
          "last_updated_time": 1629292926,
          "price": "3048.54667194"
        },
        {
          "asset": "terra1u0t35drzyy0mujj8rkdyzhe264uls4ug3wdp3x",
          "last_updated_time": 1629292926,
          "price": "27.575353646269245937"
        }
      ]
    }
  },
  "overseer_collaterals": {
    "borrower": "terra12hnhh5vtyg5juqnzm43970nh4fw42pt27nw9g9",
    "collaterals": [
      [
        "terra19mkj9nec6e3y5754tlnuz4vem7lzh4n0lc2s3l",
        "11350"
      ],
      [
        "terra1u0t35drzyy0mujj8rkdyzhe264uls4ug3wdp3x",
        "8600910"
      ]
    ]
  }
}


In [2]:
def strip_oracle_prices(oracle_prices):
    if oracle_prices['Ok'] is not None:
        return oracle_prices['Ok']['prices']
    else:
        return oracle_prices['prices']

def collaterals_total_value(overseer_collaterals, oracle_prices):
    oracle_prices = strip_oracle_prices(oracle_prices)

    collateral_address = list(map(lambda oracle: oracle['asset'], oracle_prices))
    collaterals = overseer_collaterals['collaterals']

    locked_amounts = []

    for address in collateral_address:
        collateral = next((item for item in collaterals if item[0] == address), None)
        if collateral is not None:
            locked_amounts.append(float(collateral[1]))
        else:
            locked_amounts.append(0.0)

    prices = array(list(map(lambda oracle: float(oracle['price']), oracle_prices)))
    locked_amounts = array(locked_amounts)

    print('collateral_address', collateral_address)

    print('locked_amounts', locked_amounts)
    print('prices', prices)

    locked_amounts_in_ust = locked_amounts * prices
    collateral_value = sum(locked_amounts_in_ust)

    print('locked_amounts (in UST)', locked_amounts_in_ust / 1000000.0)
    print('collateral_value (in UST)', collateral_value / 1000000.0)

    return collateral_value


print(
    'collateral_value (in UST)',
    collaterals_total_value(result['overseer_collaterals'], result['oracle_prices']) / 1000000
)


collateral_address ['terra19mkj9nec6e3y5754tlnuz4vem7lzh4n0lc2s3l', 'terra1u0t35drzyy0mujj8rkdyzhe264uls4ug3wdp3x']
locked_amounts [  11350. 8600910.]
prices [3048.54667194   27.57535365]
locked_amounts (in UST) [ 34.60100473 237.17313493]
collateral_value (in UST) 271.7741396562526
collateral_value (in UST) 271.7741396562526
